In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib

In [ ]:
# 1. Download stock data
ticker = "WBD"
df = yf.download(ticker, start="2015-01-01", end="2025-01-01")

# 2. Add features
df["Return"] = df["Close"].pct_change()
df["MA_10"] = df["Close"].rolling(10).mean()
df["MA_20"] = df["Close"].rolling(20).mean()
df["Volatility"] = df["Close"].pct_change().rolling(10).std()
df["RSI"] = 100 - (100 / (1 + (df["Return"].rolling(14).mean() / df["Return"].rolling(14).std())))
df = df.dropna()


# Features: OHLCV + Return + MAs + Volatility
feature_cols = ["Open", "High", "Low", "Close", "Volume", "Return", "MA_10", "MA_20", "Volatility","RSI"]
data = df[feature_cols].values

# Target: Next-day return
target = df["Return"].values.reshape(-1,1)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

target_scaler = MinMaxScaler(feature_range=(0,1))
scaled_target = target_scaler.fit_transform(target)

In [ ]:
X_train, y_train = [], []
window_size = 60
for i in range(window_size, len(scaled_data)):
    X_train.append(scaled_data[i-window_size:i])   # shape: (60, num_features)
    y_train.append(scaled_target[i, 0])            # predict return

X_train, y_train = np.array(X_train), np.array(y_train)
# Shape: (samples, timesteps=60, features=len(feature_cols))

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [ ]:
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1)

In [ ]:
# Save model + scalers
model.save(f"{ticker.lower()}_model.h5")
joblib.dump(scaler, f"{ticker.lower()}_scaler.pkl")
joblib.dump(target_scaler, f"{ticker.lower()}_target_scaler.pkl")

In [ ]:
def load_stock_model(ticker):
    model = load_model(f"{ticker.lower()}_model.h5")
    scaler = joblib.load(f"{ticker.lower()}_scaler.pkl")
    target_scaler = joblib.load(f"{ticker.lower()}_target_scaler.pkl")
    return model, scaler, target_scaler

In [ ]:
import numpy as np
import yfinance as yf

def predict_with_stock_model(ticker, window_size=60):
    # load model & scalers (assumes load_stock_model defined)
    try:
        model, scaler, target_scaler = load_stock_model(ticker)
    except FileNotFoundError:
        return f"Model/scaler not found for {ticker}. Train it first."
    except Exception as e:
        return f"Error loading model/scaler: {e}"

    # fetch recent data
    df = yf.download(ticker, period="6mo", interval="1d")
    if df is None or df.shape[0] == 0:
        return f"No recent data for {ticker}."

    # features same as training
    df["Return"] = df["Close"].pct_change()
    df["MA_10"] = df["Close"].rolling(window=10).mean()
    df["MA_20"] = df["Close"].rolling(window=20).mean()
    df["Volatility"] = df["Close"].pct_change().rolling(window=10).std()
    df["RSI"] = 100 - (100 / (1 + (df["Return"].rolling(14).mean() / df["Return"].rolling(14).std())))
    df = df.dropna()

    # check enough rows
    if len(df) < window_size:
        return f"Not enough data to predict (need {window_size} rows, got {len(df)})."

    # prepare features (ensure feature_cols exists in scope)
    features = df[feature_cols].values  # feature_cols defined in your training pipeline
    try:
        scaled_features = scaler.transform(features)
    except Exception as e:
        return f"Scaler transform error: {e}. Check feature order and scaler."

    # model input
    X_test = np.array([scaled_features[-window_size:]])  # shape (1, window_size, n_features)

    # prediction (model.predict returns array)
    predicted_return_scaled = model.predict(X_test)  # e.g. shape (1,1) or (1,)
    predicted_return_scaled = np.asarray(predicted_return_scaled).reshape(-1, 1)  # ensure 2D for inverse_transform

    # inverse transform safely
    try:
        predicted_return = target_scaler.inverse_transform(predicted_return_scaled)[0, 0]
    except Exception:
        # fallback: squeeze and convert to float
        predicted_return = float(np.squeeze(predicted_return_scaled))

    # ensure scalar float
    predicted_return = float(np.squeeze(predicted_return))

    # sanity checks
    if np.isnan(predicted_return) or np.isinf(predicted_return):
        return f"Model returned non-finite predicted return: {predicted_return}"

    current_price = float(df["Close"].values[-1])
    predicted_price = current_price * (1 + predicted_return)

    # decision thresholds
    if predicted_return > 0.003:   # +0.3%
      decision = "BUY ✅"
    elif predicted_return < -0.003: # -0.3%
      decision = "SELL ❌"
    else:
      decision = "HOLD ⚖️"

    return f"{ticker}: Current {current_price:.2f}, Predicted {predicted_price:.2f} (Return {predicted_return*100:.2f}%) → {decision}"

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye 👋")
        break
    try:
        response = predict_with_stock_model(user_input.upper())
        print("Chatbot:", response)
    except:
        print("Chatbot: Sorry, I don’t have a trained model for that ticker yet.")